# Prepared before Training

In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import glob
from keras.layers import Conv2DTranspose, Conv2D, Input, MaxPooling2D, concatenate, Dropout, BatchNormalization, Activation
from keras.models import Model
import random
import shutil

In [2]:
!pip install ultralytics

  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.8/914.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.whl (171 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 2.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.2 MB/s eta 0:00:00a 0:00:01
Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [15]:
from ultralytics import YOLO

# Data Preparation

## Read image files and label files

In [5]:
source_images = "/Users/thanhhuongtran/Documents/him/seed-size/dataset_close_full/images/train"
source_labels = "/Users/thanhhuongtran/Documents/him/seed-size/dataset_close_full/merged_labels/train"

In [6]:
base_dir = "dataset"
for folder in ['images/train', 'images/val', 'images/test', 
               'labels/train', 'labels/val', 'labels/test']:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

In [7]:
image_files = [f for f in os.listdir(source_images) if f.endswith(('.jpg', '.JPG', '.png', '.PNG'))]

In [9]:
len(image_files)

500

In [10]:
train_files, temp_files = train_test_split(image_files, train_size=0.7, random_state=42)
val_files, test_files = train_test_split(temp_files, train_size=0.5, random_state=42)

In [12]:
def copy_files(files, image_source, label_source, image_dest, label_dest):
    for f in files:
        # Copy image
        shutil.copy2(
            os.path.join(image_source, f),
            os.path.join(image_dest, f)
        )
        
        # Copy label (đổi đuôi file thành .txt)
        label_file = os.path.splitext(f)[0] + '.txt'
        if os.path.exists(os.path.join(label_source, label_file)):
            shutil.copy2(
                os.path.join(label_source, label_file),
                os.path.join(label_dest, label_file)
            )

In [13]:
copy_files(train_files, source_images, source_labels, 
          os.path.join(base_dir, 'images/train'), 
          os.path.join(base_dir, 'labels/train'))

copy_files(val_files, source_images, source_labels,
          os.path.join(base_dir, 'images/val'),
          os.path.join(base_dir, 'labels/val'))

copy_files(test_files, source_images, source_labels,
          os.path.join(base_dir, 'images/test'),
          os.path.join(base_dir, 'labels/test'))

In [14]:
print(f"Total images: {len(image_files)}")
print(f"Training set: {len(train_files)} images ({len(train_files)/len(image_files)*100:.1f}%)")
print(f"Validation set: {len(val_files)} images ({len(val_files)/len(image_files)*100:.1f}%)")
print(f"Test set: {len(test_files)} images ({len(test_files)/len(image_files)*100:.1f}%)")

Total images: 500
Training set: 350 images (70.0%)
Validation set: 75 images (15.0%)
Test set: 75 images (15.0%)


In [20]:
label_dirs = ["/Users/thanhhuongtran/Documents/him/seed-size/dataset/labels/train", 
              "/Users/thanhhuongtran/Documents/him/seed-size/dataset/labels/val", 
              "/Users/thanhhuongtran/Documents/him/seed-size/dataset/labels/test"]

In [21]:
def fix_label_format(label_dir):
    for file in os.listdir(label_dir):
        file_path = os.path.join(label_dir, file)
        new_lines = []
        
        with open(file_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if len(parts) > 1:
                    class_id = parts[0]  # Lấy class ID ở đầu
                    coords = parts[1:]   # Lấy tọa độ
                    new_line = " ".join(coords) + f" {class_id}\n"  # Đưa class ID xuống cuối
                    new_lines.append(new_line)

        # Ghi đè lại file với định dạng đúng
        with open(file_path, "w") as f:
            f.writelines(new_lines)

In [22]:
for label_dir in label_dirs:
    fix_label_format(label_dir)

print("Đã sửa xong định dạng file nhãn cho tất cả tập train, val, test!")

Đã sửa xong định dạng file nhãn cho tất cả tập train, val, test!


# Train the YOLOv8OBB model

In [16]:
# Load the YOLOv8-OBB model
model = YOLO("yolov8n-obb.pt")  # Use a pre-trained YOLOv8-OBB model

100%|██████████| 6.26M/6.26M [00:02<00:00, 2.26MB/s]


In [23]:
model.train(data="/Users/thanhhuongtran/Documents/him/seed-size/dataset/data.yaml", epochs=100, imgsz=640, batch=8, device="mps")

Ultralytics 8.3.72 🚀 Python-3.10.16 torch-2.6.0 MPS (Apple M1 Pro)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/Users/thanhhuongtran/Documents/him/seed-size/dataset/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

train: Scanning /Users/thanhhuongtran/Documents/him/seed-size/dataset/labels/train.cache... 350 images, 0 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:00<?, ?it/s]
val: Scanning /Users/thanhhuongtran/Documents/him/seed-size/dataset/labels/val.cache... 75 images, 0 backgrounds, 0 corrupt: 100%|██████████| 75/75 [00:00<?, ?it/s]

Plotting labels to runs/obb/train4/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/obb/train4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      6.26G     0.8841     0.6264      1.172        299        640: 100%|██████████| 44/44 [03:38<00:00,  4.97s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [00:00<?, ?it/s]

...

WARNING ⚠️ NMS time limit 2.150s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [07:32<00:00, 45.24s/it]


                   all         75       2527      0.973      0.153      0.295      0.275

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      9/100      8.61G     0.7453     0.4609      1.141        262        640: 100%|██████████| 44/44 [36:19<00:00, 49.54s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [01:00<?, ?it/s]


KeyboardInterrupt: 